## Definitions


**catalog:** refers to a Glue catalog previously created.

## Install required depenencies

In [23]:
!pip install awswrangler

## Packages

In [58]:
import awswrangler as wr
import pandas as pd
import os

## Define S3 bucket paths

In [140]:
BUCKET_NAME = 'team4-guai-artificial-intelligence-685653615554'
ROOT_BUCKET_PATH = f's3://{BUCKET_NAME}/'
INTERMEDIATE_FOLDER_NAME = '02_intermediate'
INTERMEDIATE_FOLDER_PATH = os.path.join(ROOT_BUCKET_PATH, INTERMEDIATE_FOLDER_NAME)
MODELS_FOLDER_NAME = '06_models'
MODELS_FOLDER_PATH = os.path.join(ROOT_BUCKET_PATH, MODELS_FOLDER_NAME)

## List of available datasets

In [73]:
wr.catalog.databases()

,Database,Description
0,default,
1,gdelt,
2,guai-db,
3,sdl-demo-data,


## List of available tables in a database

In [74]:
wr.catalog.tables(database='guai-db')

,Database,Table,Description,TableType,Columns,Partitions
0,guai-db,demo_intermediate,,EXTERNAL_TABLE,"productname, department, product, imageurl, da...",
1,guai-db,divipola,,EXTERNAL_TABLE,"codigodepartamento, codigomunicipio, nombredpt...",
2,guai-db,suppliers,,EXTERNAL_TABLE,"entidad, nit_entidad, departamento_entidad, ci...",


## Get location of table in S3 using catalog

In [75]:
wr.catalog.get_table_location(database='guai-db', table='divipola')

's3://team4-guai-artificial-intelligence-685653615554/01_raw/divipola/'

## Load table using catalog

In [130]:
def read_table_to_pandas(database: str, table: str, format: str='parquet'):
    """
    Read a existing table in catalog to a pandas dataframe.
    It gets the table path using the catalog.
    
    Inputs:
        database: string with the name of existing catalog database
        table: string with the name of existing catalog table
        format: str with format of table file located in s3 path
    """
    
    path = wr.catalog.get_table_location(database=database, table=table)
    if format=='parquet':
        return wr.s3.read_parquet(path=path)
    elif format=='csv':
        return wr.s3.read_csv(path=path)
    
    raise Exception("El archivo que intenta cargar no está en formato parquet o csv")


In [131]:
divipola_raw = read_table_to_pandas("guai-db", "divipola")
divipola_raw.head()

,Código Departamento,Código Municipio,nombre_dpto,nombre_mpio,tipo,cod_dpto,cod_mpio
0,5,5001,ANTIOQUIA,MEDELLÍN,Municipio,05,05001
1,5,5002,ANTIOQUIA,ABEJORRAL,Municipio,05,05002
2,5,5004,ANTIOQUIA,ABRIAQUÍ,Municipio,05,05004
3,5,5021,ANTIOQUIA,ALEJANDRÍA,Municipio,05,05021
4,5,5030,ANTIOQUIA,AMAGÁ,Municipio,05,05030


In [132]:
suppliers_raw = read_table_to_pandas("guai-db", "suppliers")
suppliers_raw.head()

,entidad,nit_entidad,departamento_entidad,ciudad_entidad,nombre_dpto,cod_dpto
0,ACOPI Antioquia,8909007481,Antioquia,Medellín,ANTIOQUIA,05
1,ADMINISTRACIÓN TEMPORAL PARA EL SECTOR AGUA PO...,892115015,La Guajira,Riohacha,LA GUAJIRA,44
2,ADMINISTRADORA COLOMBIANA DE PENSIONES COLPENS...,900336004,No Definido,No Definido,NO DEFINIDO,NO DEFINIDO
3,ADMINISTRADORA DE LOS RECURSOS DEL SISTEMA GEN...,901037916,Distrito Capital de Bogotá,No Definido,BOGOTÁ. D.C.,11
4,AEROPUERTO INTERNACIONAL MATECAÑA,891480014,No Definido,No Definido,NO DEFINIDO,NO DEFINIDO


In [133]:
wr.catalog.tables(database='sdl-demo-data')

,Database,Table,Description,TableType,Columns,Partitions
0,sdl-demo-data,compressed-parquet,,EXTERNAL_TABLE,"productname, department, product, imageurl, da...",
1,sdl-demo-data,raw,,EXTERNAL_TABLE,"productname, color, department, product, image...","partition_0, partition_1, partition_2, partiti..."


In [134]:
demo = read_table_to_pandas("sdl-demo-data", "compressed-parquet")

In [135]:
demo.head()

,productname,department,product,imageurl,date_start,date_until,price,campaign,year,month,day,hour
0,Awesome Granite Keyboard,Computers,Chips,http://lorempixel.com/640/480,Sat Apr 27 2019 09:40:11 GMT-0400 (Eastern Day...,Mon Jul 15 2019 11:10:23 GMT-0400 (Eastern Day...,45,NONE,2019,06,18,21
1,Small Granite Chair,Industrial,Salad,http://lorempixel.com/640/480,Wed Mar 06 2019 04:54:52 GMT-0500 (Eastern Sta...,Mon Jul 08 2019 05:08:00 GMT-0400 (Eastern Day...,36,BlackFriday,2019,06,18,21
2,Intelligent Cotton Keyboard,Electronics,Keyboard,http://lorempixel.com/640/480,Wed Feb 13 2019 06:32:02 GMT-0500 (Eastern Sta...,Sat Sep 14 2019 18:12:08 GMT-0400 (Eastern Day...,150,BlackFriday,2019,06,18,21
3,Refined Soft Mouse,Outdoors,Computer,http://lorempixel.com/640/480,Tue Jun 11 2019 17:59:42 GMT-0400 (Eastern Day...,Fri Oct 18 2019 09:08:06 GMT-0400 (Eastern Day...,104,10Percent,2019,06,18,21
4,Practical Cotton Salad,Baby,Salad,http://lorempixel.com/640/480,Thu Jun 06 2019 02:03:09 GMT-0400 (Eastern Day...,Fri Dec 20 2019 01:12:51 GMT-0500 (Eastern Sta...,23,10Percent,2019,06,18,21


In [137]:
def save_dataframe_to_s3_parquet(dataframe: pd.DataFrame, path: str, database: str, table: str):
    """
    Save a pandas dataframe to a parquet file in a s3 path. 
    If table in catalog already exists in the same path it is replaced
     
    Inputs:
        dataframe: pandas dataframe to be saved
        path: string containing s3 path
        database: string with name of existing database in catalog
        table: string with the existing o new name of a table to be created in catalog
        
    Returns: 
        None
    """
    
    wr.s3.to_parquet(
        df=dataframe,
        path=path,
        compression='snappy',
        database=database,
        table=table,
        dataset=True 
    )
    
save_dataframe_to_s3_parquet(dataframe=demo, path=INTERMEDIATE_FOLDER_PATH, database='guai-db', table='demo-intermediate')

In [83]:
wr.catalog.tables(database='guai-db')

,Database,Table,Description,TableType,Columns,Partitions
0,guai-db,demo_intermediate,,EXTERNAL_TABLE,"productname, department, product, imageurl, da...",
1,guai-db,divipola,,EXTERNAL_TABLE,"codigodepartamento, codigomunicipio, nombredpt...",
2,guai-db,suppliers,,EXTERNAL_TABLE,"entidad, nit_entidad, departamento_entidad, ci...",


## Serialize sklearn model

In [96]:
from sklearn import svm
from sklearn import datasets
from joblib import dump, load
import boto3

### Train model (Example)

In [92]:
clf = svm.SVC()
X, y= datasets.load_iris(return_X_y=True)
clf.fit(X, y)

SVC()

### Save Model to S3

In [138]:
def save_sklearn_model_to_s3(model, model_name):
    """
    Takes a pre-train sklean model and saves it into the "models" folder of the s3 bucket.
    To do that it has to save to sagemaker local host the file, send it to s3, and delete it from sagemaker
    
    Inputs:
        model: sklearn model to be saved
        model_name: a user assign name for the model MUST END WITH "".joblib" sufix
    
    Returns:
        None
    
    """
    dump(model, model_name) 
    s3 = boto3.resource('s3')
    s3.meta.client.upload_file(model_name, BUCKET_NAME, os.path.join(MODELS_FOLDER_NAME, model_name))
    os.remove(model_name)
    
save_sklearn_model_to_s3(clf, 'filename.joblib')

### Load Model from S3

In [142]:
def download_sklearn_model_from_s3(model_name):
    """
    -Loads a previously save sklearn model with format .joblib from s3 bucket.
    To do that it has to save the downloaded model into sagemaker localhost and delete it
    after the model has been put into an sklearn object.
    
    Inputs:
        model_name: the name of the file containing the model and located in the models folder s3 bucket
        
    Returns:
        model: the model as a sklearn object
    """
    s3 = boto3.resource('s3')
    s3.meta.client.download_file(BUCKET_NAME, os.path.join(MODELS_FOLDER_NAME, model_name), model_name)
    model = load(model_name)
    os.remove(model_name)
    return model
    
model = download_sklearn_model_from_s3('filename.joblib')